# Scraper para obtener datos públicos

In [122]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
from bs4 import BeautifulSoup

## Conexión al navegador

In [139]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
start_url = 'https://nominatransparente.rhnet.gob.mx/nomina-APF'
driver.get(start_url)

## Lectura de archivo csv para obtener nombres

In [142]:
df = pd.read_csv('ejemplo.csv') 
df['NombreCompleto'] = df.apply(lambda row: f"{row['Nombre']} {row['Paterno']} {row['Materno']}", axis=1)
df

,Nombre,Paterno,Materno,NombreCompleto
0,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
1,Luis,Cruz,Jaramillo,Luis Cruz Jaramillo
2,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
3,José Alberto,Montán,López,José Alberto Montán López
4,Andres Manuel,Lopez,Obrador,Andres Manuel Lopez Obrador
5,Juan de Dios,Olivares,Jimenez,Juan de Dios Olivares Jimenez


In [143]:
buscar = driver.find_element(By.XPATH, '/html/body/app-root/app-apf/div[2]/div[1]/div[3]/div/div[3]/a')
data = [ ]

for i in df['NombreCompleto']:
    print(i)
    #WebDriverWait(driver, 10)
    nombre = driver.find_element(By.XPATH, '//*[@id="apfName"]')
    nombre.send_keys(i)
    #WebDriverWait(driver, 10)
    buscar = driver.find_element(By.XPATH, '/html/body/app-root/app-apf/div[2]/div[1]/div[3]/div/div[3]/a')
    #WebDriverWait(driver, 10)
    buscar.click()
    
    time.sleep(5)
    # Obtener identificadores de ventanas
    ventanas = driver.window_handles
    
    
    

    # Cambiar al controlador de la ventana emergente
    

    
    try:
    # Buscar y hacer clic en el botón dentro de la ventana emergente
        driver.switch_to.window(ventanas[-1])
        ok_button = driver.find_element(By.CSS_SELECTOR, 'button.swal2-confirm.swal2-styled')
        ok_button.click()
    except NoSuchElementException:
        pass


    html_table = driver.page_source
    
    # Crear un objeto BeatifulSoup para analizar el html
    soup = BeautifulSoup(html_table, 'html.parser')
    
    # Encontrar todas las finlas de la tabla
    rows = soup.find_all('tr')
    
    for row in rows:
        cells = row.find_all('td')
        # Extraer el texto de cada celda y agregarlo a la lista de datos 
        data.append([cell.get_text(strip=True) for cell in cells])
    nombre.clear()
        
df = pd.DataFrame(data)
    
    # Asignar nombres a las columnas
df.columns = ['Número', 'Nombre', 'Institución', 'Puesto', 'Sueldo_bruto_mensual', 'Sueldo_neto_estimado' ]
print(df)
    
   
    
    

    # Ingresar nombre
    

Bryan Antonio Polito Palma
Luis Cruz Jaramillo
Bryan Antonio Polito Palma
José Alberto Montán López
Andres Manuel Lopez Obrador
Juan de Dios  Olivares Jimenez
  Número                                Nombre  \
0   None                                  None   
1  No.:1    Nombre:ANDRES MANUEL LOPEZ OBRADOR   
2   None                                  None   
3  No.:1  Nombre:JUAN DE DIOS OLIVARES JIMENEZ   

                                         Institución  \
0                                               None   
1  Institución:OFICINA DE LA PRESIDENCIA DE LA RE...   
2                                               None   
3         Institución:INSTITUTO POLITÉCNICO NACIONAL   

                                              Puesto  \
0                                               None   
1  Puesto:PRESIDENTE CONSTITUCIONAL DE LOS ESTADO...   
2                                               None   
3                       Puesto:ASISTENTE EJECUTIVO A   

               Sueldo_bruto_

In [136]:
df

,Número,Nombre,Institución,Puesto,Sueldo_bruto_mensual,Sueldo_neto_estimado
0,None,None,None,None,None,None
1,No.:1,Nombre:ANDRES MANUEL LOPEZ OBRADOR,Institución:OFICINA DE LA PRESIDENCIA DE LA RE...,Puesto:PRESIDENTE CONSTITUCIONAL DE LOS ESTADO...,"Sueldo bruto mensual:$174,026.00","Sueldo neto estimado:$120,948.07"
2,None,None,None,None,None,None
3,No.:1,Nombre:JUAN DE DIOS OLIVARES JIMENEZ,Institución:INSTITUTO POLITÉCNICO NACIONAL,Puesto:ASISTENTE EJECUTIVO A,"Sueldo bruto mensual:$10,535.76","Sueldo neto estimado:$8,428.61"
